[![ko-fi](https://ko-fi.com/img/githubbutton_sm.svg)](https://ko-fi.com/Q5Q811R5YI)  
# Apollo-Colab-Inference (Batch Processing Version) [![Open In Github](https://img.shields.io/badge/github-code-green)](https://github.com/jarredou/Apollo-Colab-Inference/)  

*Original work [Apollo: Band-sequence Modeling for High-Quality Music Restoration in Compressed Audio](https://github.com/JusperLee/Apollo)*  

The model was trained to restore/enhance lossy mp3 audio with bitrate <= 128 kbps.  
___  
### Refactored for Batch Processing:
This notebook has been modified to process all audio files in an input directory and save them to an output directory, with logging for each file.

In [ ]:
%%capture --no-stderr
#@markdown # 1. Install Dependencies & Setup Environment
%cd /content/
print("Cloning repository...")
!git clone https://github.com/JusperLee/Apollo.git && cd Apollo

print("Creating model and config directories...")
!mkdir -p /content/Apollo/model /content/Apollo/configs

print("Downloading models...")
%cd /content/Apollo/model
!wget -q 'https://huggingface.co/JusperLee/Apollo/resolve/main/pytorch_model.bin'
!wget -q 'https://huggingface.co/jarredou/lew_apollo_vocal_enhancer/resolve/main/apollo_model.ckpt'
!wget -q 'https://huggingface.co/jarredou/lew_apollo_vocal_enhancer/resolve/main/apollo_model_v2.ckpt'
!wget -q 'https://github.com/deton24/Lew-s-vocal-enhancer-for-Apollo-by-JusperLee/releases/download/uni/apollo_model_uni.ckpt'

print("Downloading config files...")
%cd /content/Apollo/configs
!wget -q 'https://huggingface.co/jarredou/lew_apollo_vocal_enhancer/resolve/main/config_apollo_vocal.yaml'
!wget -q 'https://github.com/deton24/Lew-s-vocal-enhancer-for-Apollo-by-JusperLee/releases/download/uni/config_apollo_uni.yaml'

print("Updating inference script...")
%cd /content/Apollo
!rm -f '/content/Apollo/inference.py'
!wget -q 'https://raw.githubusercontent.com/jarredou/Apollo-Colab-Inference/main/inference.py'

print("Installing required python packages...")
!pip install omegaconf ml_collections

print("Creating default input and output directories...")
!mkdir -p /content/input_audio /content/output_audio

print("Setup complete!")

In [ ]:
#@markdown # 2. Run Inference on a Directory
#@markdown ### ↓ Upload your audio files to the `input_directory` before running.
import os
import glob
import shlex
from IPython.display import clear_output

input_directory = '/content/input_audio' #@param {type:"string"}
output_directory = '/content/output_audio' #@param {type:"string"}
model = 'Lew Universal Lossy Enhancer' #@param ['MP3 Enhancer', 'Lew Vocal Enhancer', 'Lew Vocal Enhancer v2 (beta)', 'Lew Universal Lossy Enhancer']
#@markdown --- 
#@markdown #### Advanced Settings
#@markdown *For the 'Lew Universal' model, set `chunk_size` to **19**. For all other models, **25** is recommended.*
chunk_size = 19 #@param {type:"slider", min:3, max:25, step:1}
overlap = 2 #@param {type:"slider", min:2, max:10, step:1}

# Set current directory to the Apollo project folder
%cd /content/Apollo

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# --- Model and Config Selection ---
if model == 'MP3 Enhancer':
    ckpt = '/content/Apollo/model/pytorch_model.bin'
    config = 'configs/apollo.yaml'
elif model == 'Lew Vocal Enhancer':
    ckpt = '/content/Apollo/model/apollo_model.ckpt'
    config = 'configs/apollo.yaml'
elif model == 'Lew Vocal Enhancer v2 (beta)':
    ckpt = '/content/Apollo/model/apollo_model_v2.ckpt'
    config = 'configs/config_apollo_vocal.yaml'
elif model == 'Lew Universal Lossy Enhancer':
    ckpt = '/content/Apollo/model/apollo_model_uni.ckpt'
    config = 'configs/config_apollo_uni.yaml'

# --- File Discovery ---
supported_extensions = ["*.wav", "*.mp3", "*.flac", "*.m4a", "*.opus"]
audio_files = []
for ext in supported_extensions:
    search_path = os.path.join(input_directory, ext)
    audio_files.extend(glob.glob(search_path))

# --- Processing Loop ---
if not audio_files:
    print(f"❌ No audio files found in '{input_directory}'.")
    print("Please upload your files to the correct directory (use the folder icon on the left) and try again.")
else:
    total_files = len(audio_files)
    print(f"✅ Found {total_files} audio file(s) to process.")

    for i, input_file in enumerate(audio_files):
        file_name = os.path.basename(input_file)
        base_name, _ = os.path.splitext(file_name)
        output_file = os.path.join(output_directory, f"{base_name}.wav")

        print(f"\n{'='*50}")
        print(f"▶️ Processing file {i+1}/{total_files}: {file_name}")
        print(f"   -> Output will be saved as: {output_file}")

        # Build and execute the command safely
        command = (
            f"python inference.py "
            f"--in_wav {shlex.quote(input_file)} "
            f"--out_wav {shlex.quote(output_file)} "
            f"--chunk_size {chunk_size} "
            f"--overlap {overlap} "
            f"--ckpt {shlex.quote(ckpt)} "
            f"--config {shlex.quote(config)}"
        )
        
        # Using get_ipython().system() to run the shell command
        get_ipython().system(command)

        print(f"✔️ Finished processing: {file_name}")

    print(f"\n{'='*50}")
    print(f"🎉 All {total_files} files have been processed successfully! 🎉")
    print(f"Check your results in the '{output_directory}' folder.")